![title](./images/logo_nao_digital.png)

# Tema 1: Conocimientos sobre Pandas

## 1. Objetivo

Procesar en Python la información entregada por el equipo de Ingeniería de datos de Oilst de forma funcional para el análisis de los retrasos en las órdenes de los clientes.

## Datos de Oilst

Cómo se ha mencionado en el Anexo A, se ha provisto la siguiente información al equipo de `Brasil BI Consulting:`

| Archivo                      	| Descripción                                           | Formato 	|
|------------------------------	|-------------------------------------------------------------------------------------------------------------------------------------------------	  |---------	|
| olist_customers_dataset      	| Datos de identificación de los clientes               |
|                               | y la ubicación del domicilio que tienen               |
|                               |registrado con Olist para recibir pedidos.             | xlsx    	|
|
|
| olist_orders_dataset         	| Datos de identificación de las órdenes realizadas     |
|                               | por los clientes,su estatus de envío y la cronología  |
|                               | de entrega en el domicilio del cliente. 	            | csv     	|
|
|
| olist_geolocation_dataset    	| Datos de geolocalización de códigos postales          |
|                               | de zonas de Brasil.                                   | csv    |
|
|
| olist_order_items_dataset    	| Contiene la relación de artículos contenidos
|                               | en las órdenes de los clientes                        | csv     	|
|
|
| olist_order_payments_dataset 	| Contiene la relación de pagos que cada cliente hizo   |
|                               | en sus órdenes por medio de pago                      | csv     	|
|
|
| states_abreviations          	| Contiene la relación de nombres de los estados de Brasil
|                               |y sus abreviaciones.                                   | json    	|


Adicionalmente, también se tiene el diagrama de relación entre las tablas:

![title](../images/oilst_diagram.png)



Este documento se desarrollarán scripts en Python que permitan procesar la  la información de Oilst para realizar posteriormente el análisis de sus datos

## 2. Librerias de trabajo

In [3]:
# Instala libreria Pandas si no la tenemos
pip install pandas -y

SyntaxError: invalid syntax (1681458506.py, line 2)

In [1]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

/var/folders/sw/4_q3h2w132df0zbpw5zmgv_c0000gp/T/ipykernel_4323/3138929089.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 3. Lectura de datos

Primero nos encargaremos de leer los datos, indicando a Python donde se encuentra la carpeta que contiene los datos y los nombres de los archivos relevantes para el análisis.

In [2]:
#  Indicamos la ruta a la carpeta de de tu computadora 
# donde se ubican los datos del E-commerce
# Ejemplo: "C:\Usuarios\[tu nombre]\Descargas"

DATA_PATH="/Users/irvingalindo/Documents/workshops/DataEngineer/Reto1"

Ahora procederemos a definir variables que indiquen el nombre de los archivos junto con su extensión (por ejemplo, `.csv`, `.json` u otra).

In [3]:
FILE_CUSTOMERS = 'olist_customers_dataset.xlsx'
FILE_GEOLOCATIONS = 'olist_geolocation_dataset.csv'

# completa los nombres del resto de los archivos con su extesion (ejemplo .csv) ...
FILE_ITEMS = 'olist_order_items_dataset.csv'
FILE_PAYMENTS = 'olist_order_payments_dataset.csv' 
FILE_ORDERS = 'olist_orders_dataset.csv'
FILE_STATES_ABBREVIATIONS = 'states_abbreviations.json'
FILE_PROCESSED_DATA = 'oilst_processed.csv'

Echaremos mano de la utilidad `os.path.join` de Python que indicar rutas en tu computadora donde se ubican archivos, así Pandas encontrá los archivos de datos.


**Ejemplo**

A continuación mostraremos un ejemplo leyendo el archivo `olist_geolocation_dataset.csv`:

In [4]:
# Ejemplo
print(f"Ruta del archivo: {FILE_GEOLOCATIONS}")
print(os.path.join(DATA_PATH, FILE_GEOLOCATIONS))

Ruta del archivo: olist_geolocation_dataset.csv
/Users/irvingalindo/Documents/workshops/DataEngineer/Reto1/olist_geolocation_dataset.csv


In [5]:
# Leemos con pandas
geolocations = pd.read_csv(
    os.path.join(DATA_PATH, FILE_GEOLOCATIONS),
    dtype={'geolocation_zip_code_prefix': 'str'}
    )

Podemos explorar el archivo con los comandos `.head(), .info(), .describe()`

**Función .head()**

Este comando nos brinda información de los primeros renglones de la tabla:

In [6]:
geolocations.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP


**Función .describe()**

Este comando nos ayuda obtener información estadística de las variables de la tabla, entre ellas el conteo de los elementos no nulos de una columnas, la media, su desviación estándar (que es una medida de que tan dispersa se encuentra la información a partir de su media), el primer, segundo y tercer cuartil de los datos 
que se refiere a que valor de los datos ocupar el 25%, 50% y 75% de la distribución de los datos si se ordenan de menor a mayor y que se denotan, respectivamente como, denotado Q1, Q2 y Q3. Además provee información del mínimo y máximo presentes en la columna.

In [7]:
geolocations .describe()

,geolocation_lat,geolocation_lng
count,1.000163e+06,1.000163e+06
mean,-2.117615e+01,-4.639054e+01
std,5.715866e+00,4.269748e+00
min,-3.660537e+01,-1.014668e+02
25%,-2.360355e+01,-4.857317e+01
50%,-2.291938e+01,-4.663788e+01
75%,-1.997962e+01,-4.376771e+01
max,4.506593e+01,1.211054e+02


**Función .info()**

Este comando nos ayuda obtener información de cuantas entradas tiene una tabla, el tipo de dato en que se representarn sus columnas y el tamaño que ocupa en la memoria de nuestra computadora.

In [8]:
geolocations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  object 
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), object(3)
memory usage: 38.2+ MB


Ahora estamos en posición de completar el código para el resto de conjuntos de datos:

### 3.1 Archivo olist_customers_dataset

**Hint:** 

1. Pandas necesita instalar una libreria extrar para leer éste tipo de archivos (`openpyxl`).
2. Se debe expecificar el tipo de dato de `customer_zip_code_prefix` (ver Anexo A)

In [9]:
customers = pd.read_excel(
    os.path.join(DATA_PATH, FILE_CUSTOMERS),
    # Especificar el tipo de dato de customer_zip_code_prefix
    dtype={'customer_zip_code_prefix': 'str'}
    )

Verificamos que `customer_zip_code_prefix` tenga el formato correcto (Ojo: !no es un número, sino un código postal!)

In [10]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


También podemos visualizar una muestra aleatoria con el método `.sample(numero_entero)`

In [11]:
# Revisamos 10 renglones al azar
customers.sample(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
51329,5361c1b04541fb86bdb86deb42bf9d93,a7408d5a8aaa865315db143d43030537,88060,florianopolis,SC
95266,762e79437f60ec2d9007b5a3f97139a9,df1602d39699064d0cec2791acb53bb9,88106,sao jose,SC
60457,444479fd5e4914e1d03d5f3e03bf166d,9987f1d85f8bf5863cd2fd4ab034d23b,01547,sao paulo,SP
36023,bcd6cf41abb52c34fbc0c481bc144eb9,d67c054dfcbdfea828a6be99b13f8207,04363,sao paulo,SP
33782,87dd18546a67fa179880ec78244ce499,2245070e754b05a611c455c0efa0ef66,05133,sao paulo,SP
85538,6750172272a6a616067b595a9aa0d6c2,c0feca9e5d043fc29634ffe766756f11,78890,sorriso,MT
66076,ae06196c4d2b039dcdd1889bd4992f65,2ff881fd08323303966e4e983a6607ab,35368,sericita,MG
33403,78d33365650d124d2b20423fd6527ed8,a9c99a3a7e66d3a596bd9907a3079e9f,01314,sao paulo,SP
19044,23301bca57eda7af3e47546760d9e869,3e6bc4180ba32d44f8278f211e2231f7,12246,sao jose dos campos,SP
36523,6748df8bad97104336f314fed57ce74e,581a5b2be2703f51621c141635638e61,89300,mafra,SC


### 3.2 Archivo olist_order_items_dataset

In [12]:
items = pd.read_csv(
        os.path.join(DATA_PATH, FILE_ITEMS),
    )
items.sample(10)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
111624,fd86c80924b4be8fb7f58c4ecc680dae,1,d22bbc02259111e107ee7916637d4791,8d956fec2e4337affcb520f56fd8cbfd,2018-03-14 20:28:42,69.99,12.93
81622,b981e23277787bf0b0f6055ee85a22ac,1,e35b496654367ba8f9ab8974a23331e6,1da3aeb70d7989d1e6d9b0e887f97c23,2018-03-02 20:09:24,9.99,15.10
93682,d44fc0af1ec0ebbab612f2936147f717,1,8bfd18c7b7358231b2a3f9ae98eed105,79ed755314cfe6df0daef2c6cd3022cd,2017-06-21 23:25:10,179.90,17.02
21528,314309621e99dff004e43be09afbbb3c,1,378fed7e555fa94c149f571bba1d71b0,60562ab00b8054280520d390c8c0045c,2017-12-27 10:12:21,165.00,15.91
97806,dde2fb922a5e542ca84560b67901f351,1,d41e71b34a15f8bc2c7ed84561ebb600,522620dcb18a6b31cd7bdf73665113a9,2017-02-18 15:42:39,529.00,29.96
71797,a3c03356178c199a84eae0d6356d815c,1,6fe0dd040139d694869acedf3d074a08,f8db351d8c4c4c22c6835c19a46f01b0,2017-03-16 20:24:32,69.90,24.98
94893,d736ac2dcec087fda2b4ed1eb5a5032b,1,b8a0d73b2a06e7910d9864dccdb0cda2,620c87c171fb2a6dd6e8bb4dec959fc6,2018-03-06 12:20:23,59.90,25.70
30888,46194c8e92b01770503efdd7aeb49c1a,1,3e4176d545618ed02f382a3057de32b4,02f5837340d7eb4f653d676c7256523a,2017-12-18 15:31:32,129.99,17.16
57520,82e2dfceae13e4942e8e10d41fe74ec6,1,95d0a6638a6e975feca32e90c15ce526,66e0557ecc2b4dbea057e93f215f68d8,2018-02-26 20:32:30,399.00,17.55
71397,a2d8aaf39efc96fe140eda3e8f2e337e,2,f8e75ec247fd97a5e1abcaa48e9e971d,ea65d8b58316a6f2362f2a9e4b3e86ad,2018-03-07 16:55:57,37.90,11.85


Como sabemos, este conjunto contiene datos de los productos que contiene cada orden. Por ello, para el análisis nos interesará saber cual es la cantidad de productos en cada orden y el precio total de las mismas.

Esto se puede calcular mediante agregaciones de Pandas (https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.core.groupby.DataFrameGroupBy.agg.html), que basicamente nos permite hacer cálculos para un grupo en especial. En el ejemplo de inferior se muestra para cada `order_id` se cuenta la cantidad de productos (items) y el precio agregado de todos los artículos en las órdenes:

In [13]:
items_agg = items.groupby(
    ['order_id']).agg(
        # conteo de producto
        {'order_item_id': 'count',
        # suma de los precios de los artículos
        'price': 'sum'}
                      ).reset_index() 

In [14]:
items_agg.query("order_id == '18f4178d9b3c33fa79c1a0d5eb214023'")

,order_id,order_item_id,price
9620,18f4178d9b3c33fa79c1a0d5eb214023,5,224.8


Vamos a renombrar las columnas anteriores, para que sea más intuitivo su significado.

In [15]:
# Nota: el parámetro inplace sobre escribe los cambios
# en el dataframe
items_agg.rename(
    columns={'order_item_id': 'total_products', 'price': 'total_sales'},
    inplace=True
    )

In [16]:
items_agg

,order_id,total_products,total_sales
0,00010242fe8c5a6d1ba2dd792cb16214,1,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,239.90
2,000229ec398224ef6ca0657da4fc703e,1,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1,299.99
98662,fffcd46ef2263f404302a634eb57f7eb,1,350.00
98663,fffce4705a9662cd70adb13d4a31832d,1,99.90
98664,fffe18544ffabc95dfada21779c9644f,1,55.99


Vamos a repetir el proceso anterior para el resto de tablas:

### 3.3 olist_order_payments_dataset

In [17]:
payments = pd.read_csv(
    os.path.join(DATA_PATH, FILE_PAYMENTS)
    )

### 3.4 states_abbreviations

In [18]:
states_abbreviations = pd.read_json(
    os.path.join(DATA_PATH,FILE_STATES_ABBREVIATIONS)
    )

### 3.5 olist_orders_dataset

In [19]:
orders = pd.read_csv(
    os.path.join(DATA_PATH,FILE_ORDERS)
    )

Si revisamos el formato que Pandas ha otorgado a las columnas que contienen fecha encontraremos algo extraño, no se ha interpretado correctamente:

In [20]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       99441 non-null  object 
 1   customer_id                    99441 non-null  object 
 2   order_status                   99441 non-null  object 
 3   order_purchase_timestamp       99441 non-null  object 
 4   order_approved_at              99281 non-null  object 
 5   order_delivered_carrier_date   97658 non-null  object 
 6   order_delivered_customer_date  96476 non-null  object 
 7   order_estimated_delivery_date  99441 non-null  object 
 8   distance_distribution_center   96470 non-null  float64
dtypes: float64(1), object(8)
memory usage: 6.8+ MB


Por lo tanto es necesario convertir las columnas al formato de fecha `datetime` (Ver: https://towardsdatascience.com/working-with-datetime-in-pandas-dataframe-663f7af6c587).

Emplearemos la función `to_datetime` de Pandas para modificar las columnas para que se interpreten como fechas:

In [21]:
# Convierte a formato fecha completando los campos apropiados

## convierte order_purchase_timestamp
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'] , errors='coerce' )

# order_approved_at
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'] , errors='coerce' )

# order_delivered_carrier_date
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'] , errors='coerce' )

# order_delivered_customer_date
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'] , errors='coerce' ) 

# order_estimated_delivery_date
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'] , errors='coerce' )

Ahora podemos ver como ha cambiado el formato:

In [22]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   distance_distribution_center   96470 non-null  float64       
dtypes: datetime64[ns](5), float64(1), object(3)
memory usage: 6.8+ MB


Una vez corregido el tema de las fecha definiremos también algunas variables auxiliares que nos serviran posteriormente en el análisis.

Por ejemplo, es de interés conocer en que mes, año y trimestre del año sucedieron las compras, usando la columna `order_purchase_timestamp`:

In [23]:
# Define una columna con el año en que sucedió la orden
orders['year'] = orders['order_purchase_timestamp'].dt.year

# Define una columna con el mes en que sucedió la orden
orders['month'] = orders['order_purchase_timestamp'].dt.month

# Define una columna con trimestre con el que paso la orden (ej. Q12018)
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.to_period.html
orders['quarter'] = orders['order_purchase_timestamp'].dt.to_period('Q')

# Define una columna con mes y año con el que paso la orden (ej. 02-2018)
# Hint: ¿que hace el metodo ...to_period('M')?
orders['year_month'] = orders['order_purchase_timestamp'].dt.to_period('M')

orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,month,quarter,year_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,29.84,2017,10,2017Q4,2017-10
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,25.53,2018,7,2018Q3,2018-07
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,61.56,2018,8,2018Q3,2018-08
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,60.30,2017,11,2017Q4,2017-11
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,80.97,2018,2,2018Q1,2018-02


Por otro lado, también necesitamos identificar las órdenes que tuvieron retrasos prolongados. Recordemos que de acuerdo a la documentación del `Anexo A`:
* Oilst notifica el usuario de cuando llegará su pedido con el valor de la columna `order_estimated_delivery_date`,
* Además la fecha real en que se llevó la entrega se encuentra en el campo `order_delivered_customer_date`

A continuación calcularemos distancia (en días) entre ambas fecha definiendo a la variable `delta_days`:

In [24]:
# Nota: tenemos que realizar la conversion de
# segundos a días

orders['delta_days'] = (
    orders['order_delivered_customer_date'] -
    orders['order_estimated_delivery_date']
    ).dt.total_seconds()/ 60 / 60 / 24

**Pregunta:**

* ¿Porqué para convertir lo anterior a días tenemos que dividir entre 60, después entre 60 y luego entre 24? **Primero se divide entre 60 (representa los segundos de un minuto) para obtener el equivalente en minutos, al dividir el resultado de la division, nuevamente entre 60 (representa los minutos en una hora)  obtendremos el equivalente en horas, por ultimo, se divide entre 24 (el cual representa las horas que tiene un dia) el resultado para obtener los dias**

Podemos explorar ahora los valores de dicha variables con el método `.descri be()`

In [25]:
orders['delta_days'].describe()

count    96476.000000
mean       -11.179120
std         10.186113
min       -146.016123
25%        -16.244384
50%        -11.948941
75%         -6.390000
max        188.975081
Name: delta_days, dtype: float64

En el contexto del problema, los valores de `delta_days` tiene el significado:

* Un valor negativo en `delta_days` significa que el pedido llego antes de lo esperado; es decir, no existió retraso.
* Un valor de `delta_days`, mayor a 0 días pero menor a 3 días, significa que es un retrazo aceptable,
* Sin embargo, si `delta_days` es más grande que 3 días esto significa que tenemos un retrazo prolongado.

Crearemos una variable `delay_status` para indicar la discusión anterior usando el operador `where` de Numpy (https://towardsdatascience.com/creating-conditional-columns-on-pandas-with-numpy-select-and-where-methods-8ee6e2dbd5d5).

Esencialmente, el operador `where` de Numpy permite definir variables siguiendo reglas lógicas de manera condicional, similar al `if ... else ...` de Python:

In [26]:
# Define 
orders['delay_status']  = np.where(
    orders['delta_days'] > 3, 'long_delay',
    np.where(orders['delta_days'] <= 0, 'on_time','short_delay')
    )

Para ver el efecto de lo anterior podemos extraer un muestra con la función `.sample`

In [27]:
orders[['delta_days','delay_status']].sample(10)

,delta_days,delay_status
38097,-0.209433,on_time
59951,-4.484884,on_time
51018,-7.982558,on_time
8648,-10.095428,on_time
51251,-7.335718,on_time
63776,-7.510463,on_time
75018,-4.340509,on_time
19432,-16.037072,on_time
5097,-23.367130,on_time
42152,6.832245,long_delay


**Pregunta:**

* ¿Las categorías anteriores en que se han clasificado los estatus de entrega de las órdenes se pueden inteserctar? **Las tres categorias no pueden intersectarse entre si debido a que un pedido solo puede tener un tiempo de entrega. Se entrega con mucho tiempo de anterioridad (<= 0 dias), a tiempo (1, 2 ó 3 dias) o retrasado (> 3 dias)pero no dos o mas estados al mismo tiempo**


### 3.5 olist_geolocation_dataset

Aunque anteriormente hemos leído este archivo, debemos notar que contiene información redudante de muchos codigos postales, como en el caso del valor `24220`:

**Nota:** el query no contaba con las comillas simples lo cual no permitia traer la información


In [28]:
geolocations.query("geolocation_zip_code_prefix == '24220'")

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
470805,24220,-22.905817,-43.106989,niteroi,RJ
470811,24220,-22.902306,-43.112545,niteroi,RJ
470812,24220,-22.904567,-43.110491,niteroi,RJ
470820,24220,-22.902575,-43.109192,niteroi,RJ
470821,24220,-22.907500,-43.106170,niteroi,RJ
...,...,...,...,...,...
474265,24220,-22.904023,-43.111683,niteroi,RJ
474266,24220,-22.905393,-43.100512,niterói,RJ
474269,24220,-22.906420,-43.104933,niteroi,RJ
474270,24220,-22.909701,-43.108452,niteroi,RJ


Para el análisis tendremos que eliminar esta duplicaciones. Esto se puede lograr con el método `drop_duplicates`

In [29]:
unique_geolocations = geolocations.drop_duplicates(
    subset = ['geolocation_zip_code_prefix']
    )

Como se aprecia a continuación, ahora el dataframe `unique_geolocations` corrige el error:

In [30]:
unique_geolocations.query(
    "geolocation_zip_code_prefix == '24220'"
    )

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
470805,24220,-22.905817,-43.106989,niteroi,RJ


## 4. Procesamiento global

Ahora que hemos cargado a Pandas los datos del E-commerce, debemos **consolidar toda la información** en una sola tabla, lo que nos permitirá centralizar el análisis y hacer comparativos.

Para ello, nos proponemos lo siguiente:
1. A los datos de clientes le añadiremos los datos de geolocalización. **(Clientes + geolocalización)**
2. Tales datos se complementarán añadiendo los datos del nombre del estado de Brasil en que se localizan. (**Clientes + geolocalización + nombre del estado donde viven**)
3. Posteriormente archivo de órdenes, agregaremos los datos del precio y cantidad de artículos. **(Órdenes + total de artículos y precios)**
4. Finalmente, uniremos toda la información de los pasos 2 y 3 en una sola tabla.

### 4.1 Clientes + geolocalización

Para unir dos fuentes de datos, podemos usar la función `.merge` (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html). En el siguiente ejemplo se unes los datos de los clientes junto sus geolocalizaciones.

**Nota:** Los códigos postales deben tener el formato texto.

In [31]:
customers_geolocation = customers.merge(
    unique_geolocations,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)

In [32]:
customers_geolocation.sample(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
51805,346f3247d517fffb206310b3e8ed6f81,6084496782926c56bf04c556331bf884,65530,urbano santos,MA,65530,-3.209843,-43.407318,urbano santos,MA
36010,fde106d687f66d052aa9b6b169f250d0,d0ed0d8dd6a919b2aad302fa1cae682f,04379,sao paulo,SP,04379,-23.655811,-46.658434,sao paulo,SP
41411,5a85f4f7d4dcd220e4b29e0d0ae6478e,fec815965dfefb4e300b0c835045b021,74845,goiania,GO,74845,-16.721493,-49.244761,goiania,GO
56404,d5ac777fc5fcadcc6324bf3a39ef9284,4931506dc6ca25b3d8f4345d82f02b15,13231,campo limpo paulista,SP,13231,-23.199116,-46.785293,campo limpo paulista,SP
61772,62ebd1175fa5b4e5529ceabb27f72b1a,2df1c82be7fac3b14a4305bfd8de01bd,22775,rio de janeiro,RJ,22775,-22.972957,-43.397063,rio de janeiro,RJ
43994,c9f4a3e9c7d357935a0e2e930d24ae71,003e28cb6fab8a0901bff9229612375f,82560,curitiba,PR,82560,-25.398981,-49.246930,curitiba,PR
15032,5974a7436459910a0a91a5e6cdd9c6bd,6080a210886780ffc895bce6bd114e4d,91440,porto alegre,RS,91440,-30.062527,-51.146643,porto alegre,RS
77760,bf1f1943a062f996e5d32bcb5662b10f,f377461b37160ea383bbb3535714ba31,37589,silvianopolis,MG,37589,-22.030607,-45.831425,silvianopolis,MG
53260,235efb83cffa6226bb1e9083ebde2a6d,a1d88ad487531e70919ce579fae6df6f,37545,cachoeira de minas,MG,37545,-22.354227,-45.773754,cachoeira de minas,MG
56528,c3ea664ff88d7468de59ee62516b0c55,80275de537b6a389930536433e8baed3,14802,araraquara,SP,14802,-21.760806,-48.172285,araraquara,SP


### 4.2 Clientes + geolocalización + nombre del estado donde viven

Ahora repetiremos un proceso análogo pero con los nombres del estado donde viven los customers

In [33]:
# Une los dataframe customers_geolocation y states_abbreviations

customers_geolocation_estado = customers_geolocation.merge(
    states_abbreviations,
    left_on='geolocation_state',
    right_on='abbreviation',
    how='left'
)

In [34]:
customers_geolocation_estado

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,abbreviation,state_name
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.509897,-47.397866,franca,SP,SP,São Paulo
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP,09790,-23.726853,-46.545746,sao bernardo do campo,SP,SP,São Paulo
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP,01151,-23.527788,-46.660310,sao paulo,SP,SP,São Paulo
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP,08775,-23.496930,-46.185352,mogi das cruzes,SP,SP,São Paulo
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,13056,-22.987222,-47.151073,campinas,SP,SP,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,03937,sao paulo,SP,03937,-23.587901,-46.501830,são paulo,SP,SP,São Paulo
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,06764,taboao da serra,SP,06764,-23.612294,-46.765787,taboao da serra,SP,SP,São Paulo
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,60115,-3.744128,-38.510859,fortaleza,CE,CE,Ceará
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,92120,-29.956391,-51.167614,canoas,RS,RS,Rio Grande do Sul


### 4.3 Órdenes + total de artículos y precios

In [35]:
# une los dataframe orders y items_agg por order_id
orders_totals = orders.merge(
    items_agg,
    on= ['order_id'],
    how='left'
    )

In [36]:
orders_totals

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,month,quarter,year_month,delta_days,delay_status,total_products,total_sales
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,29.84,2017,10,2017Q4,2017-10,-7.107488,on_time,1.0,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,25.53,2018,7,2018Q3,2018-07,-5.355729,on_time,1.0,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,61.56,2018,8,2018Q3,2018-08,-17.245498,on_time,1.0,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,60.30,2017,11,2017Q4,2017-11,-12.980069,on_time,1.0,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,80.97,2018,2,2018Q1,2018-02,-9.238171,on_time,1.0,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,68.26,2017,3,2017Q1,2017-03,-10.369433,on_time,1.0,72.00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,14.92,2018,2,2018Q1,2018-02,-1.265324,on_time,1.0,174.90
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,36.37,2017,8,2017Q3,2017-08,-5.524803,on_time,1.0,205.99
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,32.04,2018,1,2018Q1,2018-01,-20.018819,on_time,2.0,359.98


### 4.4 Clientes + geolocalización + nombre del estado donde viven + Órdenes + total de artículos y precios

In [37]:
results = orders_totals.merge(
    customers_geolocation_estado,
    on=['customer_id'],
    how='left'
    )

### 5. Entregables

A. Realiza un script (`1_1_oilst_processed.py`)que procese la información de Oilst en generando como output un archivo en `.csv` con los datos de las siguientes columnas:

**Columns de Oilst**

* order_id,
* customer_id,
* order_status,
* order_purchase_timestamp,
* order_approved_at,
* order_delivered_carrier_date,
* order_delivered_customer_date,
* order_estimated_delivery_date,
* distance_distribution_center,
* delay_status,
* total_products,
* total_sales,
* customer_unique_id,
* customer_zip_code_prefix,
* customer_city,
* customer_state,
* geolocation_zip_code_prefix,
* geolocation_lat,
* geolocation_lng,
* geolocation_city,
* geolocation_state,
* abbreviation,
* state_name

**Columnas personalizadas en este notebook**

* year,
* month,
* quarter,
* year_month,
* delta_days
* delay_status.

B. Entregar la tabla generada en el inciso A en formato `.csv`, nombrando al archivo como `oilst_processed.csv`.

Finalmente escribiremos el resultado en un archivo separado por comas `.csv`:

In [38]:
# Completa el codigo
results.to_csv(
    os.path.join(DATA_PATH,FILE_PROCESSED_DATA), 
    # flag para no escribir el indice del dataframe al csv
    index=False
    )

In [40]:
results

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,...,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,abbreviation,state_name
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,29.84,2017,...,03149,sao paulo,SP,03149,-23.574809,-46.587471,sao paulo,SP,SP,São Paulo
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,25.53,2018,...,47813,barreiras,BA,47813,-12.169860,-44.988369,barreiras,BA,BA,Bahia
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,61.56,2018,...,75265,vianopolis,GO,75265,-16.746337,-48.514624,vianopolis,GO,GO,Goiás
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,60.30,2017,...,59296,sao goncalo do amarante,RN,59296,-5.767733,-35.275467,sao goncalo do amarante,RN,RN,Rio Grande do Norte
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,80.97,2018,...,09195,santo andre,SP,09195,-23.675037,-46.524784,santo andre,SP,SP,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,68.26,2017,...,12209,sao jose dos campos,SP,12209,-23.178732,-45.889711,são josé dos campos,SP,SP,São Paulo
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,14.92,2018,...,11722,praia grande,SP,11722,-24.001467,-46.446355,praia grande,SP,SP,São Paulo
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,36.37,2017,...,45920,nova vicosa,BA,45920,-17.891522,-39.370942,nova vicosa,BA,BA,Bahia
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,32.04,2018,...,28685,japuiba,RJ,28685,-22.555985,-42.690761,japuiba,RJ,RJ,Rio de Janeiro
